In [2]:
#ライブラリのインポート
import pandas as pd
import numpy as np
from sklearn import tree

#データセット読み込み
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

#train.head()

#データセットのサイズ確認
test_shape = test.shape
train_shape = train.shape
#print(test_shape,train_shape)

#統計量確認
#train.describe() #test.describe()

#----欠損データの確認----
def kesson_table(df):
    null_val = df.isnull().sum() #isnull()は欠損データか否かを返す
    percent = 100* df.isnull().sum()/len(df)
    kesson_table = pd.concat([null_val,percent],axis=1)#concat は連結、引数のaxisは連結方向
    #print(kesson_table)
    kesson_table_ren_columns = kesson_table.rename(
    columns = {0:"欠損数",1:"%"})
    return kesson_table_ren_columns

#Age の欠損データは平均値を代理として使う
train["Age"]=train["Age"].fillna(train["Age"].mean()) #fillnaはすべてのnanを指定の値に置き換え

#Embarkedの欠損データは最も多いSとする。
train["Embarked"] = train["Embarked"].fillna("S")

#kesson_table(train)

#カテゴリカルなデータを数字に置き換える
train["Sex"][train["Sex"] == "male"] = 0
train["Sex"][train["Sex"] == "female"] = 1
train["Embarked"][train["Embarked"] == "S" ] = 0
train["Embarked"][train["Embarked"] == "C" ] = 1
train["Embarked"][train["Embarked"] == "Q"] = 2
#train.head()

test["Sex"][test["Sex"] == "male"] = 0
test["Sex"][test["Sex"] == "female"] = 1
test["Embarked"][test["Embarked"] == "S"] = 0
test["Embarked"][test["Embarked"] == "C"] = 1
test["Embarked"][test["Embarked"] == "Q"] = 2
#test.head()

#kesson_table(test)

test["Age"] = test["Age"].fillna(test["Age"].mean())
test["Fare"] = test["Fare"].fillna(test["Fare"].mean())
#kesson_table(test)

#trainデータセットを目的変数と説明変数に分ける
target = train["Survived"].values

features_one = train[["Pclass","Age","Sex","Fare", "SibSp", "Parch", "Embarked"]].values
#testデータセットから説明変数の値を取得
test_features = test[["Pclass","Age","Sex","Fare", "SibSp", "Parch", "Embarked"]].values

#正規化を試みる200518
from sklearn import preprocessing
ss = preprocessing.StandardScaler()

features_one_std = ss.fit_transform(features_one)
test_features_std = ss.fit_transform(test_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to b

In [3]:
#公差検証
from sklearn.model_selection import cross_val_score
#SVMによる予測
from sklearn.svm import SVC

Ctune = np.arange(0.1,1.1,0.05)
gammatune = np.arange(0.1,0.3,0.01)
maxc = 0.0
maxg = 0.0
maxscore = 0.0


#パラメータの意味参考https://qiita.com/sz_dr/items/f3d6630137b184156a67
for i in Ctune:
    for j in gammatune:
        clf = SVC(kernel="rbf",C=i,gamma=j,random_state=1) #score ???? データセットを拡張して試す
        # 交差検証
        scores = cross_val_score(clf, features_one_std,target)
        # スコアの平均値
        if maxscore <= np.mean(scores):
            maxc = i
            maxg = j
            maxscore = np.mean(scores)
print(maxscore)
print(maxc)
print(maxg)

0.8327663046889713
0.7000000000000002
0.15999999999999998


In [4]:
clf = SVC(kernel="rbf",C=1.05,gamma=0.18,random_state=1)
clf.fit(features_one_std,target)
print(clf.score(features_one_std,target))

scores = cross_val_score(clf,features_one_std,target)
print(np.mean(scores))

0.8462401795735129
0.828278199736363


In [6]:
pred = clf.predict(test_features_std)

# PassengerIDを取得
PassengerId = np.array(test["PassengerId"]).astype(int)

# my_predictionとPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(pred,PassengerId,columns = ["Survived"])

# my_tree_one.csvとして出力
my_solution.to_csv("/content/my_svm_one.csv",index_label = ["PassengerId"])